## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


c:\users\abc\anaconda3\envs\deeping\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [4]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(input_shape=(32, 32, 3), kernel_size=(3,3), filters=32, activation='relu', padding='same'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

'''自己決定MaxPooling2D放在哪裡'''
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(kernel_size=(3,3),filters=32, activation='relu', padding='same'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100,activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

c:\users\abc\anaconda3\envs\deeping\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100)`
c:\users\abc\anaconda3\envs\deeping\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`


Epoch 1/100
50000/50000 [==============================] - 9s 172us/step - loss: 1.3492 - accuracy: 0.5258
Epoch 2/100
50000/50000 [==============================] - 7s 135us/step - loss: 0.9361 - accuracy: 0.6724
Epoch 3/100
50000/50000 [==============================] - 7s 132us/step - loss: 0.7648 - accuracy: 0.7321
Epoch 4/100
50000/50000 [==============================] - 6s 122us/step - loss: 0.6407 - accuracy: 0.7762
Epoch 5/100
50000/50000 [==============================] - 7s 131us/step - loss: 0.5383 - accuracy: 0.8110
Epoch 6/100
50000/50000 [==============================] - 6s 123us/step - loss: 0.4395 - accuracy: 0.8456
Epoch 7/100
50000/50000 [==============================] - 6s 126us/step - loss: 0.3559 - accuracy: 0.8766
Epoch 8/100
50000/50000 [==============================] - 6s 126us/step - loss: 0.2832 - accuracy: 0.9022
Epoch 9/100
50000/50000 [==============================] - 6s 127us/step - loss: 0.2251 - accuracy: 0.9217
Epoch 10/100
50000/50000 [===========

50000/50000 [==============================] - 7s 134us/step - loss: 0.0276 - accuracy: 0.9907
Epoch 77/100
50000/50000 [==============================] - 7s 150us/step - loss: 0.0230 - accuracy: 0.9928
Epoch 78/100
50000/50000 [==============================] - 7s 148us/step - loss: 0.0148 - accuracy: 0.9955
Epoch 79/100
50000/50000 [==============================] - 7s 137us/step - loss: 0.0189 - accuracy: 0.9943
Epoch 80/100
50000/50000 [==============================] - 7s 143us/step - loss: 0.0229 - accuracy: 0.9923
Epoch 81/100
50000/50000 [==============================] - 8s 150us/step - loss: 0.0215 - accuracy: 0.9926
Epoch 82/100
50000/50000 [==============================] - 7s 136us/step - loss: 0.0183 - accuracy: 0.9941
Epoch 83/100
50000/50000 [==============================] - 7s 137us/step - loss: 0.0175 - accuracy: 0.9942
Epoch 84/100
50000/50000 [==============================] - 7s 137us/step - loss: 0.0218 - accuracy: 0.9933
Epoch 85/100
50000/50000 [===============

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
y_pred = classifier.predict(input_example)

In [6]:
classifier.evaluate(x_test, y_test)

10000/10000 [==============================] - 1s 100us/step


[3.851839103126526, 0.6945000290870667]